### YOLO Predictions

In [1]:
# Imports
import cv2 as cv
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [2]:
# Load YAML
with open("data.yaml", mode="r") as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml["names"]
print(labels)

['Person', 'Car', 'Truck', 'Semi', 'RedLight', 'YellowLight', 'GreenLight', 'WaitLight', 'StopSign', 'SpeedLimitSign']


In [3]:
# Load YOLO Model
yolo = cv.dnn.readNetFromONNX("Model/weights/v1_best.onnx")
yolo.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [4]:
# Load an image and get predictions of the YOLO Model
img = cv.imread("test_image_1.jpg")
img_pred = img.copy()
row, col, d = img_pred.shape

# Get the YOLO prediction from the image
# First, convert image to square image
max_rc = max(row, col)
input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
input_image[0:row, 0:col] = img_pred

# Then, get predictions from square array
INPUT_WH_YOLO = 640
blob = cv.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
yolo.setInput(blob)
preds = yolo.forward() # Get predictions from YOLO Model

In [5]:
print(preds.shape)

(1, 25200, 10)


In [6]:
# Non Maximum Supression
# Step 1: Filter detections based on confidence score and probability score
CONFIDENCE_THRESHOLD = 0.4
PROBABILITY_THRESHOLD = 0.25
detections = preds[0]
boxes = []
confidences = []
classes = []

# Calculate the width and height of the input image
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] # Confidence is in the 5th column of the row
    if confidence > CONFIDENCE_THRESHOLD:
        class_score = row[5:].max() # Take the maximum probability of 10 objects possible
        class_id = row[5:].argmax() # Get the index position at which maximum probability occurs

        if class_score > PROBABILITY_THRESHOLD:
            cx, cy, w, h = row[:4]

            # Construct the bounding box from the four values
            # Get left, top, width, and height
            left = int((cx - 0.5 * w) * x_factor)
            top = int((cy - 0.5 * h) * y_factor)
            width = int(w * x_factor)
            height = int(h * y_factor)

            box = np.array([left, top, width, height])

            # Append values into respective lists
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

# Cleaning
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# Apply non-maximum suppression
indexes = cv.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.45).flatten()

In [7]:
indexes

array([41, 39, 43, 46,  6], dtype=int32)

In [8]:
# Draw the Bounding Box
for i in indexes:
    # Extract Bounding Boxes
    x, y, w, h = boxes_np[i]
    bb_conf = int(confidences_np[i] * 100)
    class_id = classes[i]
    class_name = labels[class_id]

    text = f"{class_name}: {bb_conf}%"
    
    cv.rectangle(img_pred, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv.rectangle(img_pred, (x, y - 30), (x + w, y), (255, 255, 255), -1)
    cv.putText(img_pred, text, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1)

In [9]:
cv.imshow("Original Image", img)
cv.imshow("YOLO Predictions", img_pred)
cv.waitKey(0)
cv.destroyAllWindows()

2025-01-04 20:02:49.942 Python[88581:23768336] +[IMKClient subclass]: chose IMKClient_Legacy
2025-01-04 20:02:49.942 Python[88581:23768336] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


: 